## Data cleaning

In [1]:
# Import neccessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings

warnings.filterwarnings("ignore", category = FutureWarning)
warnings.filterwarnings("ignore", category = UserWarning)

# Set display options for pandas series and dataframs
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

print("Data Cleaning and Preprocessing")
print("=" * 50)
print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")

Data Cleaning and Preprocessing
Analysis Date: 2025-09-01 14:37


### _1. Load and Document All Datasets_

In [2]:
# file paths
rwa_data_path = '../data/raw/'
processed_data_path = '../data/processed/'

# Load all neccessary datasets with error handling
try:
    orders = pd.read_csv(f'{rwa_data_path}olist_orders_dataset.csv')
    order_items = pd.read_csv(f'{rwa_data_path}olist_order_items_dataset.csv')
    customers = pd.read_csv(f'{rwa_data_path}olist_customers_dataset.csv')
    products = pd.read_csv(f'{rwa_data_path}olist_products_dataset.csv')
    sellers = pd.read_csv(f'{rwa_data_path}olist_sellers_dataset.csv')
    reviews = pd.read_csv(f'{rwa_data_path}olist_order_reviews_dataset.csv')
    payments = pd.read_csv(f'{rwa_data_path}olist_order_payments_dataset.csv')
    geolocation = pd.read_csv(f'{rwa_data_path}olist_geolocation_dataset.csv')

    print("All datasets loades successfully")
except FileNotFoundError as e:
    print(f"Error loading data: {e}")

All datasets loades successfully


### _2. Preview the Datasets_

In [3]:
def dataset_preview(dataset, dataset_name):
    """
    Data preview function
    
    Parameters:
    dataset (DataFrame): Dataset to assess
    dataset_name (str): Name of the dataset for reporting
    
    Returns:
    None
    """
    print(f"DATA PREVIEW: {dataset_name.upper()}")
    print('=' * 60)
    
    print(f"Column Names:\n{dataset.columns.to_list()}")
    print(f"\n5 Sample Rows:\n{dataset.head()}")

datasets = {
    'orders': orders,
    'order_items': order_items,
    'customers': customers,
    'products': products,
    'sellers': sellers,
    'reviews': reviews,
    'payments': payments,
    'geolocation': geolocation
}
for name, df in datasets.items():
    dataset_preview(df, name)
    print("\n\n")

DATA PREVIEW: ORDERS
Column Names:
['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']

5 Sample Rows:
                           order_id                       customer_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1  53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
2  47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   
3  949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
4  ad21c59c0840e6cb83a9ceb5573f8159  8ab97904e6daea8866dbdbc4fb7aad2c   

  order_status order_purchase_timestamp    order_approved_at  \
0    delivered      2017-10-02 10:56:33  2017-10-02 11:07:15   
1    delivered      2018-07-24 20:41:37  2018-07-26 03:24:27   
2    delivered      2018-08-08 08:38:49  2018-08-08 08:55:23   
3    delivered      2017-11-18 19:28:06  2017-11-18 19:45:59   

### _3. Assess Data Quality_

In [4]:
def assess_data_quality(dataset, dataset_name):
    """
    Data quality assessment function
    
    Parameters:
    dataset (DataFrame): Dataset to assess
    dataset_name (str): Name of the dataset for reporting
    
    Returns:
    dict: Summary of data quality metrics
    """
    print(f"DATA QUALITY REPORT: {dataset_name.upper()}")
    print('=' * 60)
    
    print(f"Shape: {dataset.shape[0]} rows x {dataset.shape[1]} columns")
    print(f"Memory usage: {dataset.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

    # Missing valuse analysis
    missing_data = dataset.isna().sum()
    missing_percent = (missing_data/len(dataset)) * 100
    missing_data_df = pd.DataFrame(
        {'Missing Count': missing_data,
         'Missing Percentage': missing_percent}).sort_values('Missing Count', ascending = False)
    if missing_data_df['Missing Count'].sum() > 0:
        print("\n Missing Valuse Found: ")
        print(missing_data_df[missing_data_df['Missing Count'] > 0])
    else:
        print("No missing valuse found.")

    # Data types
    print("\n Data Types:")
    print(dataset.dtypes.value_counts())

    # Duplicates
    duplicates = dataset.duplicated().sum()
    print(f"Duplicated rows: {duplicates}")

    return {
        'shape': dataset.shape,
        'missing_valuse': missing_data_df[missing_data_df['Missing Count'] > 0],
        'duplicates': duplicates,
        'memory_mb': dataset.memory_usage(deep=True).sum() / 1024**2
    }

datasets = {
    'orders': orders,
    'order_items': order_items,
    'customers': customers,
    'products': products,
    'sellers': sellers,
    'reviews': reviews,
    'payments': payments,
    'geolocation': geolocation
}
data_quality = {}
for name, df in datasets.items():
    data_quality[name] = assess_data_quality(df, name)
    print("\n\n")

DATA QUALITY REPORT: ORDERS
Shape: 99441 rows x 8 columns
Memory usage: 58.97 MB

 Missing Valuse Found: 
                               Missing Count  Missing Percentage
order_delivered_customer_date           2965            2.981668
order_delivered_carrier_date            1783            1.793023
order_approved_at                        160            0.160899

 Data Types:
object    8
Name: count, dtype: int64
Duplicated rows: 0



DATA QUALITY REPORT: ORDER_ITEMS
Shape: 112650 rows x 7 columns
Memory usage: 39.43 MB
No missing valuse found.

 Data Types:
object     4
float64    2
int64      1
Name: count, dtype: int64
Duplicated rows: 0



DATA QUALITY REPORT: CUSTOMERS
Shape: 99441 rows x 5 columns
Memory usage: 29.62 MB
No missing valuse found.

 Data Types:
object    4
int64     1
Name: count, dtype: int64
Duplicated rows: 0



DATA QUALITY REPORT: PRODUCTS
Shape: 32951 rows x 9 columns
Memory usage: 6.79 MB

 Missing Valuse Found: 
                            Missing Count  Mi

### _4. Data Type Conversion and optimization_

In [5]:
def downcast_int_columns(df, dataset_name):
    """
    Downcast integer columns in a DataFrame to smaller integer types to save memory.

    Parameters
    ----------
    df : pd.DataFrame
        The DataFrame containing integer columns to optimize.
    dataset_name : str
        Name of the dataset (used for display/logging purposes).

    Returns
    -------
    pd.DataFrame
        A copy of the DataFrame with integer columns downcasted.
    
    Notes
    -----
    - Only int32 and int64 columns are downcasted.
    - Prints memory usage before and after optimization.
    """
    print(f"\n🔧 DOWNCAST INT DATA TYPES: {dataset_name.upper()}")
    print("=" * 50)
    
    original_memory = df.memory_usage(deep=True).sum() / 1024**2
    print(f"Original memory usage: {original_memory:.2f} MB")
    
    # Only apply downcast to integer columns
    int_cols = df.select_dtypes(include=['int64', 'int32']).columns
    df[int_cols] = df[int_cols].apply(pd.to_numeric, downcast="integer")
    
    new_memory = df.memory_usage(deep=True).sum() / 1024**2
    print(f"New memory usage: {new_memory:.2f} MB")
    print(f"Memory reduction: {((original_memory - new_memory) / original_memory) * 100:.1f}%")

    return df

datasets = {
    'orders': orders,
    'order_items': order_items,
    'customers': customers,
    'products': products,
    'sellers': sellers,
    'reviews': reviews,
    'payments': payments,
    'geolocation': geolocation
}

for name, df in datasets.items():
    datasets[name] = downcast_int_columns(df, name) 


🔧 DOWNCAST INT DATA TYPES: ORDERS
Original memory usage: 58.97 MB
New memory usage: 58.97 MB
Memory reduction: 0.0%

🔧 DOWNCAST INT DATA TYPES: ORDER_ITEMS
Original memory usage: 39.43 MB
New memory usage: 38.68 MB
Memory reduction: 1.9%

🔧 DOWNCAST INT DATA TYPES: CUSTOMERS
Original memory usage: 29.62 MB
New memory usage: 29.24 MB
Memory reduction: 1.3%

🔧 DOWNCAST INT DATA TYPES: PRODUCTS
Original memory usage: 6.79 MB
New memory usage: 6.79 MB
Memory reduction: 0.0%

🔧 DOWNCAST INT DATA TYPES: SELLERS
Original memory usage: 0.66 MB
New memory usage: 0.65 MB
Memory reduction: 1.8%

🔧 DOWNCAST INT DATA TYPES: REVIEWS
Original memory usage: 42.75 MB
New memory usage: 42.08 MB
Memory reduction: 1.5%

🔧 DOWNCAST INT DATA TYPES: PAYMENTS
Original memory usage: 17.81 MB
New memory usage: 16.43 MB
Memory reduction: 7.8%

🔧 DOWNCAST INT DATA TYPES: GEOLOCATION
Original memory usage: 146.09 MB
New memory usage: 142.27 MB
Memory reduction: 2.6%


In [6]:
def convert_to_datetime(dataset, columns, format=None, errors='coerce'):
    """
    Convert specified columns of a DataFrame to datetime.

    Parameters
    ----------
    dataset : pd.DataFrame
        DataFrame containing the columns to convert.
    columns : list of str
        Column names to convert to datetime.
    format : str, optional
        Datetime format (default: None, auto-infer).
    errors : str, optional
        How to handle errors: 'raise', 'coerce', or 'ignore' (default: 'coerce').

    Returns
    -------
    pd.DataFrame
        A copy of the DataFrame with the specified columns converted to datetime.
    """
    dataset = dataset.copy()
    for col in columns:
        dataset[col] = pd.to_datetime(dataset[col], format=format, errors=errors)
    return dataset

In [7]:
orders_dateime_cols = ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date',
                       'order_delivered_customer_date', 'order_estimated_delivery_date']

order_items_datetime_cols = ['shipping_limit_date']
reviews_datetime_cols = ['review_creation_date', 'review_answer_timestamp']

In [8]:
# Orders date time data type conversion
print(f"Before:\n{orders[orders_dateime_cols].dtypes}")
orders_clean = convert_to_datetime(orders, orders_dateime_cols)
print(f"\nAfter:\n{orders_clean[orders_dateime_cols].dtypes}")

Before:
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
dtype: object

After:
order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object


In [14]:
# Order_items date time data type conversion
print(f"Before:\n{order_items[order_items_datetime_cols].dtypes}")
order_items_clean = convert_to_datetime(order_items, order_items_datetime_cols)
print(f"\nAfter:\n{orders_items_clean[order_items_datetime_cols].dtypes}")

Before:
shipping_limit_date    object
dtype: object

After:
shipping_limit_date    datetime64[ns]
dtype: object


In [10]:
# reviews date time data type conversion
print(f"Before:\n{reviews[reviews_datetime_cols].dtypes}")
reviews_clean = convert_to_datetime(reviews, reviews_datetime_cols)
print(f"\nAfter:\n{reviews_clean[reviews_datetime_cols].dtypes}")

Before:
review_creation_date       object
review_answer_timestamp    object
dtype: object

After:
review_creation_date       datetime64[ns]
review_answer_timestamp    datetime64[ns]
dtype: object


### _5. Handling Missing Values_

We only have missing datas from two tables (orders and reviews)

In [11]:
# Clean Orders dataset
print(f"Before: {orders_clean.isna().sum().sum()} NaNs")
orders_clean = orders_clean.dropna(subset = ['order_approved_at'])
orders_clean = orders_clean.fillna({'order_delivered_customer_date': 'unkown', 'order_delivered_carrier_date': 'unknown'})
print(f"After: {orders_clean.isna().sum().sum()} NaNs")

Before: 4908 NaNs
After: 0 NaNs


In [12]:
# Clean Reviews dataset
print(f"Before: {reviews_clean.isna().sum().sum()} NaNs")
reviews_clean  = reviews_clean.fillna({'review_comment_title': 'No Title', 'review_comment_message': 'No Message'})
print(f"After: {reviews_clean.isna().sum().sum()} NaNs")

Before: 145903 NaNs
After: 0 NaNs


### _6. Creating Master Dataset_

In [19]:
def create_master_dataset(orders_df, items_df, customers_df, reviews_df, payments_df):
    """
    Create a master dataset by intelligently joining related tables
    """
    print("\nCREATING MASTER DATASET")
    print("=" * 30)

    # Start with orders as the base
    master = orders_df.copy()
    print(f"Starting with orders: {len(master)} records")

    # Adding customer information
    master = master.merge(customers_df, on = 'customer_id', how = 'left')
    print(f"After adding customers: {len(master)} records")

    # Add order items
    items_agg = order_items.groupby('order_id').agg({
        'price': ['sum', 'count', 'mean'],
        'freight_value': 'sum',
        'product_id': 'nunique'
    }).round(2)

    # Flatten column names
    items_agg.columns = ['_'.join(col).strip() for col in items_agg.columns.values]
    items_agg = items_agg.reset_index()

    master = master.merge(items_agg, on = 'order_id', how = 'left')
    print(f"After adding order items: {len(master)} records")

    # Add reviews scores
    reviews_agg = reviews_df.groupby('order_id').agg({
        'review_score': 'mean'
    }).round(2).reset_index()
    
    master = master.merge(reviews_agg, on='order_id', how='left')
    print(f"After adding reviews: {len(master)} records")
    
    # Add payment information
    payments_agg = payments_df.groupby('order_id').agg({
        'payment_value': 'sum',
        'payment_installments': 'max'
    }).reset_index()
    
    master = master.merge(payments_agg, on='order_id', how='left')
    print(f"Final master dataset: {len(master)} records")
    
    return master

# Create the master dataset
master_dataset = create_master_dataset(
    orders_clean, order_items_clean, customers, 
    reviews_clean, payments.copy()
)


CREATING MASTER DATASET
Starting with orders: 99281 records
After adding customers: 99281 records
After adding order items: 99281 records
After adding reviews: 99281 records
Final master dataset: 99281 records


### _7. Save Processed data_

In [25]:
print("Saving Processed Data")
print('=' * 60)

datasets_to_save = {
    'orders_clean': orders_clean,
    'customers_clean': customers,
    'order_items_clean': order_items_clean,
    'reviews_clean': reviews_clean,
    'master_dataset': master_dataset
}

for name, df in datasets_to_save.items():
    filepath = f'{processed_data_path}{name}.csv'
    df.to_csv(filepath, index=False)
    print(f"Saved {name}: {df.shape[0]:,} rows × {df.shape[1]} columns")

print(f"\n All processed data saved to: {processed_data_path}")

Saving Processed Data
Saved orders_clean: 99,281 rows × 8 columns
Saved customers_clean: 99,441 rows × 5 columns
Saved order_items_clean: 112,650 rows × 7 columns
Saved reviews_clean: 99,224 rows × 7 columns
Saved master_dataset: 99,281 rows × 20 columns

 All processed data saved to: ../data/processed/
